In [100]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import os

import folium
from folium.plugins import MarkerCluster
import matplotlib.cm as cm
import matplotlib.colors as colors
# KMeans 
def KMeans_cluster(df, num_clusters, covariate):
    # Fit the model
    model = KMeans(num_clusters, random_state=123)
    df["cluster"] = model.fit_predict(df[covariate])

    # calculate the count and the location for each cluster
    cluster_centers = []
    for label in range(num_clusters):
        cluster_points = df[df['cluster'] == label]
        centroid = cluster_points[covariate].mean().tolist()
        cluster_centers.append((centroid, label))
    centers = pd.DataFrame(cluster_centers)
    centers = centers.rename(columns = {0:"location", 1:"cluster"})
    number_of_occurences = pd.DataFrame(df['cluster'].value_counts())
    number_of_occurences.reset_index(inplace=True)
    number_of_occurences.columns = ['cluster', 'Count']
    number_of_occurences = number_of_occurences.merge(centers)
    number_of_occurences = number_of_occurences.drop_duplicates("cluster")

    return number_of_occurences


In [102]:
# KMeans for intervention occurrences
df = pd.read_csv('intervention_info.csv')
death = df[df['Abandon reason']=='Overleden']
spatial = ['Latitude intervention', 'Longitude intervention']
intervention_cluster = KMeans_cluster(df, 50, spatial)

# KMeans for aed locations
aed = pd.read_csv('AED_info.csv')
spatial = ['Latitude AED', 'Longitude AED']
aed_cluster = KMeans_cluster(aed, 50, spatial)

optimal_aed = pd.read_csv('best_aed_locations_genetics.csv')
spatial = ['Latitude', 'Longitude']
optimal_aed_cluster = KMeans_cluster(optimal_aed, 50, spatial)

/Users/peie/anaconda3/envs/MDA/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/peie/anaconda3/envs/MDA/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/peie/anaconda3/envs/MDA/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [65]:
# KMeans for dead cases
spatial = ['Latitude intervention', 'Longitude intervention']
death_cluster = KMeans_cluster(death, 50, spatial)

/Users/peie/anaconda3/envs/MDA/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/var/folders/xn/lctj27s9109d7xx92bv542580000gn/T/ipykernel_30278/2081430704.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cluster"] = model.fit_predict(df[covariate])


In [101]:
# Visualization clusters
center = (50.8503, 4.3517)
figure = folium.Map(center, zoom_start=12)


#intervention_group = folium.FeatureGroup(name='Intervention Cases', control=True)

#death_group = folium.FeatureGroup(name='Death Cases', control=True)

aed_group = folium.FeatureGroup(name='AED Current Locations', control=True)

optimal_aed_group = folium.FeatureGroup(name='AED Optimal Locations', control = True)

def visual_cluster(number_of_occurences,bound_color,fill_color,event, layer):
    global figure
    n_mean = number_of_occurences['Count'].mean()
    for location, number, cluster in zip(number_of_occurences['location'],
                                               number_of_occurences['Count'],
                                               number_of_occurences['cluster'],):
        folium.vector_layers.CircleMarker(
            [location[0], location[1]],
            radius=number/n_mean*2, 
            color=bound_color,
            fill=True,
            fill_color=fill_color,
            fill_opacity=0.6,
            tooltip = str(number)+','+'- Cluster ' + str(cluster) + '-' + event
        ).add_to(layer)
    
    figure.add_child(layer)

#visual_cluster(intervention_cluster,'blue','blue','Intervention',intervention_group)
visual_cluster(aed_cluster,'green','green','AED',aed_group)
#visual_cluster(death_cluster,'red','red','Death',death_group)
visual_cluster(optimal_aed_cluster,'blue','blue','Optimal AED',optimal_aed_group)
folium.LayerControl().add_to(figure)
figure.save('aed_reallocation.html')

In [103]:
# Visualization clusters
center = (50.8503, 4.3517)
figure = folium.Map(center, zoom_start=8)

intervention_group = folium.FeatureGroup(name='Intervention Cases', control=True)
death_group = folium.FeatureGroup(name='Death Cases', control=True)


def visual_cluster(df,location_columns,bound_color,fill_color,event,layer):
    global figure
    # n_mean = number_of_occurences['Count'].mean()
    marker_cluster = MarkerCluster().add_to(layer)
    for location in zip(df[location_columns[0]],df[location_columns[1]]):
        folium.vector_layers.Marker(
            [location[0], location[1]],
            #radius=number/n_mean*3, 
            color=bound_color,
            fill=True,
            fill_color=fill_color,
            fill_opacity=0.6,
            tooltip = event,
            icon=folium.Icon(color=fill_color)
        ).add_to(marker_cluster)

    figure.add_child(layer)

visual_cluster(df,['Latitude intervention', 'Longitude intervention'], 'blue','blue','Intervention',intervention_group)
visual_cluster(death,['Latitude intervention', 'Longitude intervention'], 'red','red','Death',death_group)

folium.LayerControl().add_to(figure)
figure.save('map_intervention.html')

In [99]:
center = (50.8503, 4.3517)
figure = folium.Map(center, zoom_start=9)

aed_group = folium.FeatureGroup(name='AED Current Locations', control=True)
# optimal_aed_group = folium.FeatureGroup(name='AED Optimal Locations', control = True)


visual_cluster(aed,['Latitude AED', 'Longitude AED'],'green','green','AED',aed_group)
#visual_cluster(optimal_aed,['Latitude', 'Longitude'],'white','white','Optimal AED',optimal_aed_group)

figure.save('map_AED.html')